#Installing Dependencies

In [1]:
!wget -O - https://apt.llvm.org/llvm-snapshot.gpg.key | apt-key add -
!apt-add-repository "deb http://apt.llvm.org/jammy/ llvm-toolchain-jammy-16 main"
!apt update
!apt install -y llvm-16 llvm-16-dev clang-16 cmake build-essential


--2025-05-26 00:18:40--  https://apt.llvm.org/llvm-snapshot.gpg.key
Resolving apt.llvm.org (apt.llvm.org)... 199.232.194.49, 199.232.198.49, 2a04:4e42:4c::561, ...
Connecting to apt.llvm.org (apt.llvm.org)|199.232.194.49|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3145 (3.1K) [application/octet-stream]
Saving to: ‘STDOUT’

-                   100%[===================>]   3.07K  --.-KB/s    in 0s      

2025-05-26 00:18:40 (78.9 MB/s) - written to stdout [3145/3145]

OK
Repository: 'deb http://apt.llvm.org/jammy/ llvm-toolchain-jammy-16 main'
Description:
Archive for codename: llvm-toolchain-jammy-16 components: main
More info: http://apt.llvm.org/jammy/
Adding repository.
Press [ENTER] to continue or Ctrl-c to cancel.
Adding deb entry to /etc/apt/sources.list.d/archive_uri-http_apt_llvm_org_jammy_-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/archive_uri-http_apt_llvm_org_jammy_-jammy.list
Get:2 https://cloud.r-project.org/bin/linux/u

##Setting the CUDA PATH


In [2]:
import os
os.environ["CUDA_PATH"]="/usr/local/cuda"

In [22]:
!export PATH=/usr/lib/llvm-16/bin:$PATH


##Uploading the CUDA File for Matrix Multiplication

In [3]:
from google.colab import files
uploaded=files.upload()

Saving normal_matmul.cu to normal_matmul.cu


#Converting the CUDA source code to IR form

## Compiling .cu to .ll

In [4]:
!clang-16 -x cuda --cuda-path=$CUDA_PATH --cuda-gpu-arch=sm_75 -S -emit-llvm normal_matmul.cu

clang: warning: CUDA version is newer than the latest partially supported version 11.8 [-Wunknown-cuda-version]


#Uploading LLVM Pass for Optimization

In [5]:
uploaded=files.upload()

Saving warp_optimization.cpp to warp_optimization.cpp


# Uploading LLVM Pass Header File

In [6]:
uploaded=files.upload()

Saving warp_optimization.h to warp_optimization.h


## Upload Driver File

## With the help of driver file the .ll file is converted to optimized.ll file

In [7]:
uploaded=files.upload()

Saving driver.cpp to driver.cpp


# Upload CMake

In [8]:
upload_file=files.upload()

Saving CMakeLists.txt to CMakeLists.txt


#Build

## The build creates the executable file warp_opt_driver

In [10]:
!cmake -DLLVM_DIR=/usr/lib/llvm-16/cmake/llvm . && make


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test HAVE_FFI_CALL
-- Performing Test HAVE_FFI_CALL - Success
-- Found FFI: /usr/lib/x86_64-linux-gnu/libffi.so
-- Performing Test Terminfo_LINKABLE
-- Performing Test Terminfo_LINKABLE - Success
-- Found Terminfo: /usr/lib/x86_64-linux-gnu/libtinfo.so
-- Found ZLIB: /usr/lib/x86_64-linux-gnu/libz.so (found version "1.2.11")
-- Found zstd: /usr/lib/x86_64-linux-gnu/libzstd.so
-- Found LibXml2: /usr/lib/x86_64-linux-gnu/libxml2.so (found version "2.9.13")
-- Found CURL: /usr/li

## Run LLVM Pass

##Upon executing warp_opt_driver the optimized.ll file is created

In [11]:
!./warp_opt_driver

# Compiling .ll file to .ptx file

## For normal_matmul.ll

In [37]:
!clang-16 -x ir normal_matmul.ll --cuda-path=$CUDA_PATH -nocudalib -nocudainc --cuda-gpu-arch=sm_75 -S -o normal_matmul.ptx

clang: warning: argument unused during compilation: '-nocudainc' [-Wunused-command-line-argument]
clang: warning: argument unused during compilation: '--cuda-gpu-arch=sm_75' [-Wunused-command-line-argument]


## For optimized_matmul.ll

In [39]:
!clang-16 -x ir optimized_matmul.ll --cuda-path=$CUDA_PATH -nocudalib -nocudainc --cuda-gpu-arch=sm_75 -S -o optimized_matmul.ptx

clang: warning: argument unused during compilation: '-nocudainc' [-Wunused-command-line-argument]
clang: warning: argument unused during compilation: '--cuda-gpu-arch=sm_75' [-Wunused-command-line-argument]
